# Neo4j relation table 생성

In [1]:
import pandas as pd
import ast
from neo4j import GraphDatabase
import psycopg2

In [2]:
# server table 확인
connection_info = "host=147.47.200.145 dbname=teamdb16 user=team16 password=qwer16 port=34543"
conn = psycopg2.connect(connection_info)

try:
    # 테이블을 Pandas.Dataframe으로 추출
    df = pd.read_sql('SELECT * FROM citation_data',conn)

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

/tmp/ipykernel_2595715/4016362387.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM citation_data',conn)


In [2]:
# df = pd.read_csv('../Jaewon/output.csv')

In [113]:
# df = df.drop('Unnamed: 0', axis=1)
# df

In [3]:
df[['id','refer_doc']]

,id,refer_doc
0,53e9abc9b7602d970357cb38,"{53e9a9beb7602d970331e620,558a419fe4b031bae1f6..."
1,53e9abc9b7602d970357cd02,"{53e999adb7602d97021f3a16,53e99ac4b7602d970234..."
2,53e9abc9b7602d970357dedf,"{558a324184ae84d265bc29d3,53e9aeddb7602d970390..."
3,53e9abc9b7602d970357e448,"{53e998e1b7602d9702121035,53e99d9db7602d970265..."
4,53e9abcab7602d9703580418,"{53e9bb29b7602d97047665fa,53e997b5b7602d9701f9..."
...,...,...
644795,53e9abc4b7602d970357849c,"{53e9989bb7602d97020d53ec,53e99dfeb7602d97026c..."
644796,53e9abc5b7602d9703578c05,"{53e99a0eb7602d970225fc55,53e99f4fb7602d970282..."
644797,53e9abc9b7602d970357bb6b,"{53e99905b7602d97021409af,53e9a3a5b7602d9702cb..."
644798,53e9abc9b7602d970357bc4c,"{53e998fdb7602d970213b2d7,53e99e54b7602d970271..."


In [112]:
df['fos'].value_counts()

fos
Mathematics              187476
Distributed computing    141508
Machine learning         110524
The Internet              40778
Embedded system           33484
Multimedia                30551
Scalability               30016
Computation               29394
Cloud computing           21278
Database                  19791
Name: count, dtype: int64

## node에 저장할 키워드

In [5]:
keyword_dict = {}
for idx, row in df.iterrows():
    keywords = ast.literal_eval(row['keywords'])
    for keyword in keywords:
        keyword  = keyword.lower().strip()
        keyword_dict[keyword] = keyword_dict.get(keyword, 0) + 1

In [6]:
len(keyword_dict)

2130925

In [7]:
len({k:v for k,v in keyword_dict.items() if v>=50})

11971

In [8]:
{k:v for k,v in keyword_dict.items() if v>=50}.keys()

dict_keys(['computational complexity', 'memory management', 'type inference', 'hardness of approximation', 'front end', 'dynamic logic', 'expected utility', 'case base reasoning', 'cubic spline', 'closed-form expression', 'blind source separation', 'higher order statistics', 'independent component analysis', 'optimisation', 'ica', 'jade', 'convergence property', 'distributed computing', 'cache memory', 'computational modeling', 'kernel', 'complexity', 'stability', 'decoding', 'error statistics', 'capacity region', 'error probability', 'channel coding', 'encoding', 'measurement', 'vectors', 'channel capacity', 'awgn channels', 'maximum likelihood decoding', 'additive white gaussian noise channel', 'markov processes', 'ad hoc networks', 'probability', 'queueing theory', 'statistical analysis', 'telecommunication channels', 'telecommunication services', 'telecommunication traffic', 'wireless lan', 'channel access', 'network traffic', 'steady state', 'spread spectrum communication', 'kalma

In [9]:
top_keywords = list({k:v for k,v in keyword_dict.items() if v>=50}.keys())

In [18]:
pd.DataFrame(top_keywords, columns=['top_keyword']).to_csv('top_keyword.csv', index=False)

In [10]:
temp_list = []
for idx, row in df.iterrows():
    paper_id = row['id']
    keywords = ast.literal_eval(row['keywords'])
    for keyword in keywords:
        if keyword in top_keywords:
            temp_list.append([paper_id, keyword])
    
paper_keyword = pd.DataFrame(temp_list, columns = ['id','top_keyword'])

In [11]:
paper_keyword

,id,keyword
0,53e99804b7602d9702016027,computational complexity
1,53e99804b7602d9702016027,memory management
2,53e99804b7602d9702016027,type inference
3,53e99804b7602d9702016027,hardness of approximation
4,53e99804b7602d9702016027,front end
...,...,...
2772501,53e9b79fb7602d970434a71b,gaussian processes
2772502,53e9b79fb7602d970434a71b,gaussian noise
2772503,53e9b79fb7602d970434a71b,feedback linearization
2772504,53e9b79fb7602d970434a71b,feedback


In [12]:
paper_keyword.to_csv('paper_keyword.csv', index=False)

In [5]:
paper_keyword = pd.read_csv('paper_keyword.csv')
paper_keyword

,id,keyword
0,53e99804b7602d9702016027,computational complexity
1,53e99804b7602d9702016027,memory management
2,53e99804b7602d9702016027,type inference
3,53e99804b7602d9702016027,hardness of approximation
4,53e99804b7602d9702016027,front end
...,...,...
2772501,53e9b79fb7602d970434a71b,gaussian processes
2772502,53e9b79fb7602d970434a71b,gaussian noise
2772503,53e9b79fb7602d970434a71b,feedback linearization
2772504,53e9b79fb7602d970434a71b,feedback


In [10]:
temp = paper_keyword[paper_keyword['keyword']=='65h10']
temp

,id,keyword
2438243,53e9ac42b7602d970360bc3f,65h10
2462530,53e9ae69b7602d9703886760,65h10
2593894,53e9baa6b7602d97046d1489,65h10
2598209,53e9bb0eb7602d9704746b54,65h10
2650067,555040e945ce0a409eb377b5,65h10
2653760,555043a345ce0a409eb4888f,65h10
2653821,555043a845ce0a409eb48ab0,65h10


In [11]:
merged = pd.merge(df, temp, left_on='id', right_on='id', how='inner')
merged

,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract,keyword
0,53e9ac42b7602d970360bc3f,A class of parallel hybrid two-stage iteration...,Applied Mathematics and Computation,1999,"['matrix multisplitting', 'cr:g1.3', 'parallel...",Mathematics,"['53e9aa6db7602d97033e1661', '53e9b655b7602d97...",4,Journal,A class of parallel hybrid two-stage iteration...,65h10
1,53e9ae69b7602d9703886760,A numerical algorithm for the solution of a ph...,J. Comput. Physics,2010,"['65z05', 'important solution invariant', '35q...",Mathematics,"['53e9a366b7602d9702c74f19', '53e9b600b7602d97...",2,Journal,We describe an algorithm for the numerical sol...,65h10
2,53e9baa6b7602d97046d1489,Efficient constraint dynamics using MILC SHAKE,J. Comput. Physics,2008,"['70e60', '90c53', '70-08', 'important case', ...",Mathematics,"['53e99ea0b7602d9702768144', '53e9a291b7602d97...",21,Journal,We address the problem of imposing rigid const...,65h10
3,53e9bb0eb7602d9704746b54,A new approach to continuation methods for com...,Oper. Res. Lett.,1997,"['p 0 -functions', 'new continuation method', ...",Mathematics,"['53e998cdb7602d9702106cf5', '53e99e28b7602d97...",50,Journal,We design a new continuation method for the so...,65h10
4,555040e945ce0a409eb377b5,Newton Methods to Solve a System of Nonlinear ...,J. Optimization Theory and Applications,2015,"['newton iterations', '34d20', 'convergence', ...",Mathematics,"['53e9ad41b7602d9703722137', '53e9b634b7602d97...",2,Journal,Fundamental insight into the solution of syste...,65h10
5,555043a345ce0a409eb4888f,On the numerical solution of a class of nonsta...,Journal of Computational and Applied Mathematics,2014,"['15b48', 'nonlinear problems', '34b15', 'nume...",Mathematics,"['53e9b451b7602d9703f5078a', '555041dd45ce0a40...",3,Journal,The paper deals with the numerical solution of...,65h10
6,555043a845ce0a409eb48ab0,Extended convergence results for the Newton-Ka...,J. Computational Applied Mathematics,2015,"['65g99', 'analytic operator', 'newton-kantoro...",Mathematics,"['53e9bcd2b7602d9704955a9d', '53e9a957b7602d97...",1,Journal,We present new semilocal and local convergence...,65h10


In [15]:
merged.loc[3,'title']

'A new approach to continuation methods for complementarity problems with uniform P-functions'

In [14]:
for idx, row in merged.iterrows():
    print(row['keywords'])

['matrix multisplitting', 'cr:g1.3', 'parallel hybrid two-stage iteration', 'accelerated overrelaxation technique', 'convergence', 'block bordered system of linear equations', 'linear system', 'two-stage iteration method', '65h10', '65w05', 'linear equations', 'iteration method']
['65z05', 'important solution invariant', '35q99', '65m12', '65m20', 'newton step', 'bdf algorithm', '65f10', 'pfm system', '65m06', 'polycrystalline material', 'newton-krylov algorithm', 'generalized minimum residual algorithm', 'jacobian linear system', 'phase-field model', 'polycrystalline microstructure', 'nonlinear system', 'newton–krylov methods', 'local order parameter', 'numerical algorithm', 'local orientation', '65h10', 'method of lines', 'system of equations', 'microstructures', 'linear system', 'materials science', 'projection method']
['70e60', '90c53', '70-08', 'important case', 'computational method', 'constraint matrix', 'lagrange multipliers', 'molecular dynamics', 'rattle', 'simulations', 'ef

## citation graph edge

In [115]:
df

,id,title,venue,year,keywords,fos,refer_doc,n_citation,doc_type,abstract
0,53e9af94b7602d97039dd8b5,A Situation Calculus-based Approach To Model U...,Computing Research Repository,2003,"{""markov property"",satisfiability,""situation c...",Machine learning,"{53e99a3cb7602d970229d140,53e9a31fb7602d9702c2...",11,Journal,This paper presents an augmented situation c...
1,53e9af94b7602d97039ddb53,A User Profiling Architecture for Textual-Base...,"Inteligencia Artificial, Revista Iberoamerican...",2003,"{personalization,""intelligent agents"",""user pr...",Machine learning,"{53e9ab20b7602d97034a5c28,53e999cab7602d970220...",11,Journal,Several intelligent agents have been developed...
2,53e9af94b7602d97039dd707,Mining fuzzy association rules from questionna...,Knowl.-Based Syst.,2009,"{""fuzzy sets"",""data mining"",""questionnaire dat...",Machine learning,"{5582cbfd0cf2a81de26651ca,53e99915b7602d970214...",98,Journal,Association rule mining is one of most popular...
3,53e9af94b7602d97039ddc91,An Integrative Neural Network with Feedback Co...,IJCNN,2006,"{""learning (artificial intelligence)"",""pattern...",Machine learning,"{53e9aa9cb7602d9703415186,53e99b51b7602d97023f...",2,Conference,This paper presents a novel integrative neural...
4,53e9af94b7602d97039ddb9a,Data mining for intelligent structure form sel...,PAKDD Workshops,2007,"{""high rise case base"",""fuzzy interval divisio...",Machine learning,{53e9a7eab7602d9703128855},1,Conference,This paper presents a uniform model for high-r...
...,...,...,...,...,...,...,...,...,...,...
644795,53e9af94b7602d97039dcb2a,An alternative approach for modelling and simu...,Simulation Modelling Practice and Theory,2004,"{""Neural network"",""Traffic model problem"",Iden...",Machine learning,{53e99991b7602d97021d2b83},30,Journal,It is assumed that there is a complicated rela...
644796,53e9af94b7602d97039dc83e,Fuzzy linear and nonlinear discriminant analys...,Intell. Data Anal.,2009,"{""fuzzy logic"",""nonlinear discriminant analysi...",Machine learning,"{53e9ad33b7602d970371729e,53e9990db7602d970214...",2,Journal,This paper presents two novels algorithms base...
644797,53e9af94b7602d97039dce17,A visual framework to understand similarity qu...,IJBIDM,2010,"{""new proposed structure"",""share dynamic"",""exi...",Machine learning,"{53e9a8a9b7602d97031f6116,53e998e9b7602d970212...",2,Journal,This paper presents the MAMView framework to h...
644798,53e9af94b7602d97039dce6f,A multi-level thresholding-based method to lea...,CIARP,2011,"{""data warehouse"",""fuzzy measure"",""genetic alg...",Machine learning,"{558ac63284ae84d265bf8f5f,53e99a4eb7602d97022a...",0,Conference,Learn fuzzy membership functions automatically...


In [116]:
temp_list = []
for idx, row in df.iterrows():
    paper_id = row['id']
    refer_docs = row['refer_doc'].replace('{',"['").replace('}',"']").replace(',',"','")
    refer_docs = ast.literal_eval(refer_docs)
    for refer_doc in refer_docs:
        temp_list.append([paper_id, refer_doc])
    
paper_ref = pd.DataFrame(temp_list, columns = ['id','ref_doc'])

In [117]:
paper_ref

,id,ref_doc
0,53e9af94b7602d97039dd8b5,53e99a3cb7602d970229d140
1,53e9af94b7602d97039dd8b5,53e9a31fb7602d9702c289ad
2,53e9af94b7602d97039dd8b5,53e9a3d4b7602d9702ce537f
3,53e9af94b7602d97039dd8b5,53e9a953b7602d97032ab560
4,53e9af94b7602d97039dd8b5,53e9ae62b7602d970387bc76
...,...,...
8236570,53e9af94b7602d97039dce6f,558a61ae84ae84d265bd4564
8236571,53e9af94b7602d97039dce6f,558ac63284ae84d265bf8f5f
8236572,53e9af94b7602d97039dd1b4,53e9a123b7602d9702a12eff
8236573,53e9af94b7602d97039dd1b4,53e9a81fb7602d9703162a51


In [118]:
paper_ref.to_csv('paper_ref.csv', index=False)

## Neo4j Query

In [ ]:
CREATE CONSTRAINT UniqueId FOR (p:Paper) REQUIRE p.id IS UNIQUE;

CALL apoc.load.jdbc("jdbc:postgresql://localhost:34543/teamdb16?user=team16&password=qwer16","citation_data") YIELD row MERGE (:Paper {id:row.id,venue:row.venue,title:row.title,citation:toInteger(row.n_citation)});

CALL apoc.load.jdbc("jdbc:postgresql://localhost:34543/teamdb16?user=team16&password=qwer16","paper_ref") YIELD row MATCH (p1:Paper {id:row.id}) MATCH (p2:Paper {id:row.ref_doc}) MERGE (p1)-[rel:REF]->(p2) RETURN count(rel);

MATCH (:Paper {title:"OLAP for Trajectories"})--(p:Paper) RETURN p;

MATCH (:Paper {title:"OLAP for Trajectories"})-[rel:REF*1..2]-(p:Paper) RETURN p;

In [ ]:
CALL gds.graph.project.cypher(
  'citations',
  'MATCH (:Paper {title:"OLAP for Trajectories"})-[rel:REF*1..2]-(p:Paper) RETURN id(p) as id',
  'MATCH (p1:Paper {title:"OLAP for Trajectories"})-[rel:REF*1..2]-(p2:Paper) RETURN id(p1) AS source, id(p2) AS target')
YIELD
  graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels;
    
CALL gds.articleRank.stream('citations')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).title AS title, score
ORDER BY score DESC, title ASC

CALL gds.graph.drop('citations', false) YIELD graphName;

In [5]:
# neo4j 드라이버 import 및 함수 정의
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [132]:
def graph_recommendation(title_list, k=5):
    dbname = "teamdb16"
    uri_param = "bolt://147.47.200.145:37687"
    user_param = "team16"
    pwd_param = "bdai2023"
    conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)
    
    score_dict = {}
    for title in title_list:
        # subgraph 초기화
        cypher = '''CALL gds.graph.drop('citations', false) YIELD graphName'''
        conn.query(cypher, db=dbname)
        
        # edge 3개 이내 그래프 저장
        cypher = f'''CALL gds.graph.project.cypher("citations","MATCH (:Paper {{title:'{title}'}})-[rel:REF*1..2]-(p:Paper) RETURN id(p) as id","MATCH (p1:Paper {{title:'{title}'}})-[rel:REF*1..2]-(p2:Paper) RETURN id(p1) AS source, id(p2) AS target") YIELD graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels;'''
        conn.query(cypher, db=dbname)
        
        # article rank 계산
        cypher = '''CALL gds.articleRank.stream('citations') YIELD nodeId, score RETURN gds.util.asNode(nodeId).title AS title, score ORDER BY score DESC, title ASC'''
        response = conn.query(cypher, db=dbname)
        
        if not response:
            continue
            
        for i in response:
            score_dict[i['title']] = score_dict.get(i['title'], 0) + i['score']
            
    conn.close()     
    df = pd.DataFrame(score_dict.items(), columns=['title','score'])
    df = df[~df['title'].isin(title_list)]
    df = df.sort_values('score', ascending=False, ignore_index=True)
    return df

In [153]:
result = graph_recommendation(['OLAP for Trajectories',
                               'A Data Model for Moving Objects Supporting Aggregation',
                               'Spatial aggregation: Data model and implementation'])
result

,title,score
0,Piet: a GIS-OLAP implementation,0.462695
1,Spatiotemporal aggregate computation: a survey,0.461743
2,Efficient OLAP Operations in Spatial Data Ware...,0.460953
3,Indexing Spatio-Temporal Data Warehouses,0.459628
4,Spatial hierarchy and OLAP-favored search in s...,0.459169
...,...,...
392,Monitoring Streams - A New Class of Data Manag...,0.150416
393,Mining place-matching patterns from spatio-tem...,0.150416
394,Mining long sequential patterns in a noisy env...,0.150416
395,Mining good sliding window for positive pathog...,0.150416


In [134]:
result['title'][2]

'Efficient OLAP Operations in Spatial Data Warehouses'

In [149]:
graph_recommendation(['Spatial aggregation: Data model and implementation'])

,title,score
0,Spatiotemporal aggregate computation: a survey,0.154275
1,Spatial hierarchy and OLAP-favored search in s...,0.153847
2,Piet: a GIS-OLAP implementation,0.153847
3,Efficient OLAP Operations in Spatial Data Ware...,0.153420
4,A Data Model for Moving Objects Supporting Agg...,0.152992
...,...,...
175,Determining the Impact Regions of Competing Op...,0.150427
176,Detecting summarizability in OLAP,0.150427
177,Denormalization strategies for data retrieval ...,0.150427
178,Data Warehouse Performance: Selected Technique...,0.150427


In [135]:
graph_recommendation(['Efficient OLAP Operations in Spatial Data Warehouses'])

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project.cypher`: Caused by: org.neo4j.graphdb.NotInTransactionException: The transaction has been closed.}
Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.articleRank.stream`: Caused by: java.util.NoSuchElementException: Graph with name `citations` does not exist on database `teamdb16`. It might exist on another database.}


,title,score


In [154]:
temp = df[(df['n_citation']>100)&(df['fos']=='Machine learning')].sample(5)
temp['title'].values

array(['A Flexible Content Adaptation System Using a Rule-Based Approach',
       'Random Features for Large-Scale Kernel Machines',
       'A systematic review of software fault prediction studies',
       'Learning to Recognize Activities from the Wrong View Point',
       'Soft-to-Hard Vector Quantization for End-to-End Learning Compressible Representations.'],
      dtype=object)

In [157]:
graph_recommendation(['A systematic review of software fault prediction studies'])

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project.cypher`: Caused by: org.neo4j.graphdb.NotInTransactionException: The transaction has been closed.}
Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.articleRank.stream`: Caused by: java.util.NoSuchElementException: Graph with name `citations` does not exist on database `teamdb16`. It might exist on another database.}


,title,score
